# TLDR

Here we work through Google's interactive GKE[^GKE] Autopilot Walkthrough. 

[^GKE]: Google Kubernetes Engine

# Resources

- [GKE Modes of Operation](https://cloud.google.com/kubernetes-engine/docs/concepts/choose-cluster-mode)
- [Kubernetes Engine -> Create an Autopilot cluser](https://console.cloud.google.com/kubernetes/auto/add)

# About GKE Modes of Operation

GKE allows you to select 2 modes of operation when creating a cluser:

- Autopilot
- Standard

# Creating a Cluster in Autopilot Mode

Creating an Autopilot cluster benefits from Google Cloud handling the following:

- **Nodes**: Automated node provisioning, scaling, and maintenance
- **Networking**: VPC-native traffic routing for clusters
- **Security**: Shielded GKE Nodes and Workload Identity
- **Telemetry**: Cloud Operations logging and monitoring

## Setup Cluster Basics

The following CLI command is comparible to what Google Cloud runs to create the Kubernetes cluster. 

To select the region, I used Google's [Cloud Region Picker](https://cloud.withgoogle.com/region-picker/).

```bash
gcloud beta container \
    --project \
"gke-autopilot-cluster-479822" clusters create-auto "my-autopilot-cluster" \
    --region \
"us-east1" \
    --release-channel \
"regular" \
    --enable-ip-access \
    --no-enable-google-cloud-access \
    --network \
"projects/gke-autopilot-cluster-479822/global/networks/default" \
    --subnetwork \
"projects/gke-autopilot-cluster-479822/regions/us-east1/subnetworks/default" \
    --cluster-ipv4-cidr \
"/17" \
    --binauthz-evaluation-mode=DISABLED
```

## Fleet Registration

This feature helps you to manage multiple clusters together.

A fleet lets you logically group and normalize Kubernetes clusters, helping you uplevel management from individual clusters to groups of clusters. To use multi-cluster capabilities and apply consistent policies across your systems, register your cluster to a fleet.

- [GKE Fleet Management Overview](https://cloud.google.com/kubernetes-engine/fleet-management/docs)

## Networking

This section is for defining access to the Control Pane. You can select access through using DNS or through using IPv4 addresses.

You also define cluster networking, e.g. how applications in the cluster communicate with each other & how clients can reach them.

## Advanced Settings

Here you can choose from 3 target Release Channels: Rapid, Regular, & Stable. The default is Regular (recommended). 

- Rapid
  - Use to test new releases before they're qualified for production use or general availability.
- Regular
  - Balance feature availability and release stability. Versions have been qualified over a longer period. Recommended for most users.
- Stable
  - Use it to prioritize stability over new features. GKE rolls out changes and new versions on the Stable channel last, after validating them on the Rapid and Regular channels.

Learn more about choosing the best channel:

- [Choosing the Best Channel for your Cluster](https://cloud.google.com/kubernetes-engine/docs/concepts/release-channels#what_channel_should_i_use)

Once selected, default configuration can be changed & defined for the following areas:

- Automation, Service Mesh, Backup plan, Security, AI and Machine Learning, Operations, Metadata

## Review & Create

Here your can review all settings, and then click Create Cluster. After about 5 minutes, your cluster should be up & running. 

# Configure Access to Cluster 

Here we walk through configuring the Kubernetes command line tool to access our cluster. This will allow us to monitor/interact with our cluster on our local machine. 

Resources:

- [Install kubectl and configure cluster access](https://docs.cloud.google.com/kubernetes-engine/docs/how-to/cluster-access-for-kubectl)

Firstly, [gcloud](https://docs.cloud.google.com/sdk/docs/install-sdk) CLI must be installed. This allows us to work with the [Google Cloud SDK](https://docs.cloud.google.com/sdk/docs/overview) from the command line / terminal. 

## `gcloud` Setup & Config

This is my first time using Google Cloud, so I need to configure it using:

```bash
gcloud init --no-launch-browser
```

After authenticating through the browser, next we will pick a project to use. Here I select the project that Google Cloud created when setting up the autpilot cluster: `gke-autopilot-cluster-479822`.

Next, we specify a default Google Compute Engine region-zone to use. I selected `us-east1-b` here since it's best for my physical location.

After this, Google SDK should have created a `~/.config/gcloud/` directory with your configuration. The Google Cloud SDK configuration information is written to `~/.config/gcloud/configurations/config_default`. This looks something like the following:

```
[core]
account = <replace-with-your-email>@gmail.com
project = gke-autopilot-cluster-479822

[compute]
zone = us-east1-b
region = us-east1
```

## `kubectl` Config

We can use `gcloud` to generate a "***kubeconfig***" context that stores our GKE cluster information for `kubectl`.

This kubeconfig file is stored under `$HOME/.kube/config` on macOS. 

```bash
gcloud container clusters get-credentials my-autopilot-cluster --location=us-east1
```

Expected output:

```
Fetching cluster endpoint and auth data. 
kubeconfig entry generated for my-autopilot-cluster.
```

The kubectl configuration can be output with `kubectl config view`.

To view current context of kubectl, run `kubectl config current-context`, output:

```
gke_gke-autopilot-cluster-479822_us-east1_my-autopilot-cluster
```

For more commonly used `kubectl` commands, see:

- [kubectl Quick Reference](https://kubernetes.io/docs/reference/kubectl/quick-reference/)

# Deploying Application to Autopilot Cluster

The next sections of the tutorial involve deploying a microservice demo application to your Autopilot Cluster. 

The walkthrough on <https://console.cloud.google.com> has you use the Google Cloud Console built-in terminal & Cloud Shell (VS Code) editor.

However, Kubernetes was designed to be able to do this remotely, so we will use the `kubectl` CLI in our local dev environment. With API access/auth & config setup, these steps can be done in either your local dev setup or within the Google Cloud Console. 

Additionally, VS Code's Kuberenetes extension allows us to monitor our cluster. We'll show what that looks like as well.

Here we clone the demo microservice application (an Online Boutique). Then, we deploy it to our autopilot cluster.

```bash
git clone --depth 1 https://github.com/GoogleCloudPlatform/microservices-demo.git
cd microservices-demo
kubectl apply -f ./release/kubernetes-manifests.yaml
```

The output for `kubectl apply` below:

```{bash}
echo hello
```

```{.bash}
echo hello
deployment.apps/currencyservice created
service/currencyservice created
serviceaccount/currencyservice created
Warning: autopilot-default-resources-mutator:Autopilot updated Deployment default/loadgenerator: defaulted unspecified 'cpu' resource for containers [frontend-check] (see http://g.co/gke/autopilot-defaults).
deployment.apps/loadgenerator created
serviceaccount/loadgenerator created
deployment.apps/productcatalogservice created
service/productcatalogservice created
serviceaccount/productcatalogservice created
deployment.apps/checkoutservice created
service/checkoutservice created
serviceaccount/checkoutservice created
deployment.apps/shippingservice created
service/shippingservice created
serviceaccount/shippingservice created
deployment.apps/cartservice created
service/cartservice created
serviceaccount/cartservice created
deployment.apps/redis-cart created
service/redis-cart created
deployment.apps/emailservice created
service/emailservice created
serviceaccount/emailservice created
deployment.apps/paymentservice created
service/paymentservice created
serviceaccount/paymentservice created
deployment.apps/frontend created
service/frontend created
service/frontend-external created
serviceaccount/frontend created
deployment.apps/recommendationservice created
service/recommendationservice created
serviceaccount/recommendationservice created
deployment.apps/adservice created
service/adservice created
serviceaccount/adservice created
```

Next, we can monitor our cluster. Below shows us the state 